# import dependencies

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains.retrieval import create_retrieval_chain
from typing import List
import os
from dotenv import load_dotenv

/Users/sumanyadav/Desktop/PROJECT/Lernix/backend/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise EnvironmentError("OPENAI_API_KEY environment variable not set")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Chunk text

In [3]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        length_function=len
    )
    return splitter.split_text(text)

# Convert chunks to Documents

In [4]:
def convert_to_document(chunks: List[str]) -> List[Document]:
    return [Document(page_content=chunk) for chunk in chunks]



# Create Hybrid Retriever

In [5]:
def create_retriever(docs: List[Document]):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    dense_vectorstore = FAISS.from_documents(docs, embeddings)
    dense_retriever = dense_vectorstore.as_retriever(search_kwargs={"k": 10})

    sparse_retriever = BM25Retriever.from_documents(docs)
    sparse_retriever.k = 3

    return EnsembleRetriever(
        retrievers=[dense_retriever, sparse_retriever],
        weights=[0.7, 0.3]
    )

## build mcq generation rag chain

In [6]:
def build_mcq_chain(retriever, num_questions: int = 5, difficulty: str = "medium"):
    # Format the prompt with fixed values at chain creation time
    prompt_text = f"""Based on the following content, generate {num_questions} multiple choice questions.

    Requirements:
    - Difficulty level: {difficulty}
    - Each question should have 4 answer choices (A, B, C, D)
    - Indicate the correct answer
    - Questions should cover different topics from the content
    - Avoid yes/no questions
    
    Format each question as:
    Question X: [question text]
    A) [option A]
    B) [option B]
    C) [option C]
    D) [option D]
    Correct Answer: [letter]
    Explanation: [brief explanation]

    Content:
    {{context}}
    """
    
    prompt = PromptTemplate.from_template(prompt_text)

    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7
    )

    document_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt
    )

    return create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=document_chain
    )

# format each question on a separate line

In [10]:
def format_mcqs_detailed(text: str) -> str:
    """
    Ensure each question starts on a new line with proper spacing
    """
    # Normalize line endings
    text = text.replace("\r\n", "\n").strip()

    # Ensure a blank line before every Question except the first
    text = text.replace("\nQuestion", "\n\nQuestion")

    return text


## MCQ generation function

In [11]:
def generate_mcqs(text: str, num_questions: int = 5, difficulty: str = "medium") -> str:
    chunks = chunk_text(text, chunk_size=800, overlap=100)
    docs = convert_to_document(chunks)
    retriever = create_retriever(docs)
    mcq_chain = build_mcq_chain(retriever, num_questions, difficulty)

    result = mcq_chain.invoke({
        "input": "Generate multiple choice questions from the document"
    })

    # Format the output
    formatted_result = format_mcqs_detailed(result["answer"])
    
    return formatted_result

In [9]:
generate_mcqs("""
Interpersonal Communication with StrangersInterpersonal communication with strangers refers to the exchange of information, thoughts, andfeelings between people who do not know each other well. This type of communication commonlyoccurs in daily life, such as when speaking with classmates, coworkers, shopkeepers, serviceproviders, or people met for the first time. Effective communication with strangers requires clarity,politeness, and respect. Since there is little or no prior relationship, first impressions are veryimportant. Simple actions like maintaining eye contact, using a friendly tone, and practicing goodlistening skills help create a positive interaction. Clear language and appropriate body posture alsoreduce misunderstandings. Active listening plays a key role in interpersonal communication. Itinvolves paying attention, avoiding interruptions, and responding thoughtfully. When communicatingwith strangers, asking relevant questions and acknowledging their responses shows interest andrespect, helping build trust quickly. Nonverbal communication is equally important. Facialexpressions, gestures, and personal space can strongly influence how a message is received.Being aware of cultural differences is essential, as gestures and communication styles may varyacross cultures. Safety and boundaries should always be maintained while interacting withstrangers. Personal information should not be shared unnecessarily, and communication shouldremain appropriate and professional. Overall, effective interpersonal communication with strangershelps develop social skills, build confidence, and create positive connections in everyday life.
""")

'Question 1: What is the primary focus of interpersonal communication with strangers?  \nA) Establishing long-term relationships  \nB) Exchanging information, thoughts, and feelings  \nC) Avoiding any form of communication  \nD) Conducting formal business transactions  \nCorrect Answer: B  \nExplanation: Interpersonal communication with strangers primarily involves the exchange of information, thoughts, and feelings between individuals who do not know each other well.\n\n\nQuestion 2: Which of the following is NOT a recommended practice for effective communication with strangers?  \nA) Maintaining eye contact  \nB) Using a friendly tone  \nC) Interrupting frequently  \nD) Practicing good listening skills  \nCorrect Answer: C  \nExplanation: Interrupting frequently is not a recommended practice for effective communication, as it can disrupt the flow of conversation and show a lack of respect.\n\n\nQuestion 3: How does active listening contribute to interpersonal communication with stran